# Example Usage: Derivative Estimation Toolkit

This notebook shows how to compute numerical derivatives using the STEM method and the five-point stencil method.
No plotting is done here — just functional evaluation and comparison.


In [1]:
import numpy as np

from derivkit import HybridDerivativeCalculator


In [2]:
# Define a test function
f = lambda x: 3 * x**2 + np.sin(x)

# Point at which to evaluate the derivative
x0 = 1.0

# Initialize the calculator
hdc = HybridDerivativeCalculator(f, x0)


In [3]:
# Compute derivative using the five-point stencil method
stencil_result = hdc.five_point_stencil_method()

# Compute derivative using the STEM method
stem_result = hdc.stem_method()

# Print the results
print(f"Five-Point Stencil Result at x = {x0:.2f}: {stencil_result:.6f}")
print(f"STEM Method Result at x = {x0:.2f}: {stem_result:.6f}")


Five-Point Stencil Result at x = 1.00: 6.540302
STEM Method Result at x = 1.00: 6.539653


## Using STEM Method for 2D Cosmology Data Vectors

In many cosmology applications, the observable data vector (e.g. angular power spectra $C_\ell$) is multi-dimensional.
For example, a typical $C_\ell$ vector might have shape `(number of ells, number of spectra)` such as shear-shear,
shear-position, and position-position correlations.

To compute the derivative of this 2D data vector with respect to a parameter (e.g. $\Omega_m$),
we flatten the array to 1D before passing it to the `HybridDerivativeCalculator`.
After computing the derivative, we reshape it back to the original 2D structure for interpretability.
This derivative matrix tells us how each component of the data vector changes with small variations in the input parameter.


In [4]:
# Simulate a fake angular power spectrum C_ell data vector
num_ells = 20  # number of multipoles
num_spectra = 3  # e.g. shear-shear, shear-position, position-position
ells = np.linspace(100, 2000, num_ells)

# Define a fake function of Omega_m that returns a 2D (ells x spectra) power spectrum
def fake_cl_function(omega_m):
    # Toy model: C_ell ∝ Omega_m * f(ell), where f(ell) varies by component
    return np.array([
        omega_m * np.log(ell + 1) * (1 + i * 0.1)
        for i in range(num_spectra)
        for ell in ells
    ]).reshape(num_ells, num_spectra)

# Flatten the output since the derivative calculator expects a 1D array
def wrapped_flattened_function(omega_m):
    return fake_cl_function(omega_m).flatten()

# Central value of Omega_m
omega_m_central = 0.3

# Initialize the derivative calculator
calc = HybridDerivativeCalculator(wrapped_flattened_function, omega_m_central)

# Compute the derivative
stem_derivative = calc.stem_method()

# Reshape the derivative back to original shape for interpretability
stem_derivative_reshaped = stem_derivative.reshape(num_ells, num_spectra)

# Print results
print("STEM derivative with respect to Omega_m:\n")
for i in range(num_spectra):
    print(f"Component {i + 1}:")
    print(stem_derivative_reshaped[:, i])
    print("-" * 40)



STEM derivative with respect to Omega_m:

Component 1:
[4.61512052 5.99396143 6.55250789 6.90875478 7.17088848 7.37838371
 7.55013534 5.8336354  6.83826671 7.35444704 7.70437155 7.9694357
 8.18286875 8.36154257 6.84853232 7.67831392 8.16420631 8.50909179
 8.7766642  8.99531681]
----------------------------------------
Component 2:
[5.30330491 6.2166061  6.68586095 7.00397414 7.24494155 7.43897159
 7.60140233 6.27782129 7.03845443 7.48385578 7.8000008  8.04527551
 8.24570708 5.53814462 7.19275371 7.86300946 8.29050574 8.60506617
 8.85406046 9.06016241]
----------------------------------------
Component 3:
[5.70711026 6.39859493 6.80350526 7.09090982 7.31388683 7.49609735
 5.07663257 6.59335757 7.20775868 7.59963026 7.88797733 8.11622208
 8.30514888 6.36396589 7.45992732 8.02303314 8.40476896 8.69392986
 8.92676591 9.1216828 ]
----------------------------------------
